In [1]:
from lxml import etree
import pandas as pd
from collections import OrderedDict
import re
pd.options.mode.chained_assignment = None
from datetime import datetime, date
import xlrd

In [2]:
FN_MGAE = "March 2016 All Fields MGAE Export.xls"
FN_WB = "worldbank-ht-20160326.xml"
FN_CA = "canada-ht-20160326.xml"
FY_START_MONTH = 10 # Starts on 1 October
RELEVANT_FYs = [2010,2011,2012,2013,2014,2015]
COLS = ['EAMM ID', 'Project Title', 'Budget Code Project ID', 'Cris Number', 'Organizations and Project ID', 'Project Comments', 'Objective', 'Project Description', 'Results', 'Status', 'Actual Approval Date', 'Actual Completion Date', 'Actual Start Date', 'Proposed Approval Date', 'Proposed Completion Date', 'Proposed Start Date', 'Country', 'District', 'Implementation Level', 'Region', 'Zone', 'Donor Agency', 'Donor Group', 'Donor Type', 'Financing Instrument', 'Funding Organization ID', 'Multi Donor', 'Type Of Assistance', 'Actors', 'Issues', 'Measures Taken', 'Primary Sector', 'Primary Sector sub-sector', 'Primary Sector Sub-Sub-Sector', 'Secondary Sector', 'Secondary Sector sub-sector', 'Responsible Organization', 'Line Ministry Groups', 'Executing Agency', 'Executing Agency Groups', 'Executing Agency Type', 'Implementing Agency', 'Implementing Agency Groups', 'Implementing Agency Type', 'Contracting agency', 'Contracting Agency Acronym', 'Contracting Agency Department/Division', 'Contracting Agency Groups', 'Beneficiary Agency', 'Beneficiary Agency  Department/Division', 'Beneficiary Agency Groups', 'Sector Group', 'Sector Group Department/Division', 'National Planning Objectives', 'National Planning Objectives Level 1', 'National Planning Objectives Level 2', 'National Planning Objectives Level 3', 'Primary program', 'Primary Program Level 1', 'Primary Program Level 2', 'Primary Program Level 3', 'Secondary program', 'Secondary Program Level 1', 'Secondary Program Level 2', 'Secondary Program Level 3', 'Financial Instrument', 'FY', 'Ministry Code', 'On/Off/Treasury Budget', 'Project Code', 'Proposed Project Amount']
FIN_TYPES = ['Actual Commitments', 'Actual Disbursements', 'Planned Commitments', 'Planned disbursements']
FIN_GROUPS = ['FY2010_11', 'FY2011_12', 'FY2012_13', 'FY2013_14', 'FY2014_15', 'FY2015_16', 'Total']
for f in FIN_GROUPS:
    for t in FIN_TYPES:
        COLS.append("%s_%s" % (f, t))

In [3]:
doc_wb = etree.parse(FN_WB)
doc_ca = etree.parse(FN_CA)

### Set up some helper functions

In [4]:
# Extract data from IATI activities
nsmap = {"xml": "http://www.w3.org/XML/1998/namespace"}
def relevant_FY(transaction, relevant_FYs=RELEVANT_FYs):
    if transaction["FY"] in relevant_FYs: return True
    return False

def sum_transactions_in_FY(transactions, FY):
    def this_relevant_FY(transaction):
        if transaction["FY"] == FY: return True
        return False
    
    relevant_transactions = filter(this_relevant_FY, transactions)
    return sum(map(lambda x: x["value"], relevant_transactions))

def sum_FYs(dataframe):
    for FY in RELEVANT_FYs:
        dataframe["iati_disbursements_%s" % FY] = dataframe["transactions"].apply(lambda x: sum_transactions_in_FY(x, FY))
    dataframe = dataframe.drop("transactions", 1)
    return dataframe

# Handle v2.01 data
def get_transactions(transactions):
    def get_FY(date):
        d = datetime.strptime(date, "%Y-%m-%d")
        year = d.year
        if d.month < FY_START_MONTH: return year-1
        return year
    
    for transaction in transactions:
        yield {"date": unicode(transaction.find("transaction-date").get("iso-date")),
               "value": float(transaction.find("value").text),
               "FY": get_FY(unicode(transaction.find("transaction-date").get("iso-date")))
              }
def get_data(activity):
    return OrderedDict({
        "iati_identifier": activity.find("iati-identifier").text,
        "title": unicode(activity.find("title/narrative[@xml:lang='en']", namespaces=nsmap).text),
        "start_date": unicode(activity.find("activity-date[@type='2']").get("iso-date")),
        "aid_type": unicode(activity.find("default-aid-type").get("code")),
        "activity_status": unicode(activity.find("activity-status").get("code")),
        "haiti_country_pct": float(activity.find("recipient-country[@code='HT']").get("percentage")),
        "implementing_org": unicode(activity.find(
                    "participating-org[@role='4']/narrative[@xml:lang='en']",
                    namespaces=nsmap
                    ).text),
        "iati_currency": unicode(activity.get("default-currency")),
        "iati_total_disbursements": float(activity.xpath("sum(transaction[transaction-type/@code='3']/value/text())")),
        "iati_total_commitments": float(activity.xpath("sum(transaction[transaction-type/@code='2']/value/text())")),
        "transactions": filter(relevant_FY, get_transactions(activity.xpath("transaction[transaction-type/@code='3']")))
    })
def parse_activities(activities):
    for activity in activities:
        yield get_data(activity)

## We have to handle v1.05 data differently...
def get_data_105(activity):
    return OrderedDict({
        "iati_identifier": activity.find("iati-identifier").text,
        "title": unicode(activity.find("title").text),
        "start_date": unicode(activity.find("activity-date[@type='start-planned']").get("iso-date")),
        "aid_type": unicode(activity.find("default-aid-type").get("code")),
        "activity_status": unicode(activity.find("activity-status").get("code")),
        "haiti_country_pct": float(activity.find("recipient-country[@code='HT']").get("percentage")),
        "implementing_org": unicode(activity.find(
                    "participating-org[@role='Implementing']"
                    ).text),
        "iati_currency": unicode(activity.get("default-currency")),
        "iati_total_disbursements": float(activity.xpath("sum(transaction[transaction-type/@code='D']/value/text())")),
        "iati_total_commitments": float(activity.xpath("sum(transaction[transaction-type/@code='C']/value/text())")),
        "transactions": filter(relevant_FY, get_transactions(activity.xpath("transaction[transaction-type/@code='D']")))
    })
def parse_activities_105(activities):
    for activity in activities:
        yield get_data_105(activity)

# Get project ID from MGAE data
def get_project_ID(value):
    try:
        if re.search("^ - (.*) \((.*)\)\n - (.*) \((.*)\)", value):
            result = re.search("^ - (.*) \((.*)\)\n - (.*) \((.*)\)", value).groups()
            return result[0], result[2]
        elif re.search("^\((.*)\) \((.*)\)", value):
            return re.search("^\((.*)\) \((.*)\)", value).groups()[0]
        elif re.search("^\((.*)\)", value):
            return re.search("^\((.*)\)", value).groups()[0]
        elif re.search("^(.*) \((.*)\)", value):
            return re.search("^(.*) \((.*)\)", value).groups()[0]
        else:
            return value
    except TypeError: 
        return value
def get_iati_id(row):
    prefixes = {"Banque mondiale": "44000",
                "Canada": "CA-3"}
    project_id = re.sub("-", "", str(row["Project ID"]))
    return "%s-%s" % (prefixes[row["Donor Group"]], project_id)

In [5]:
### Prepare IATI data

In [6]:
wb_iati_data = pd.DataFrame(parse_activities_105(doc_wb.xpath("//iati-activity")))
wb_iati_data = wb_iati_data.set_index("iati_identifier")
wb_iati_data = sum_FYs(wb_iati_data)

ca_iati_data = pd.DataFrame(parse_activities(doc_ca.xpath("//iati-activity")))
ca_iati_data = ca_iati_data.set_index("iati_identifier")
ca_iati_data = sum_FYs(ca_iati_data)

In [7]:
iati = pd.concat([wb_iati_data, ca_iati_data])

### Prepare MGAE data

In [8]:
d = pd.read_excel(FN_MGAE, skiprows=range(0,8), header=None, names=COLS)

In [9]:
mgae = pd.DataFrame(d)
mgae = mgae[["Project Title", "Organizations and Project ID", "Donor Agency", "Donor Group", 'Proposed Project Amount', 'FY2010_11_Actual Commitments',
 'FY2010_11_Actual Disbursements', 'FY2010_11_Planned Commitments', 'FY2010_11_Planned disbursements', 'FY2011_12_Actual Commitments',
 'FY2011_12_Actual Disbursements', 'FY2011_12_Planned Commitments', 'FY2011_12_Planned disbursements', 'FY2012_13_Actual Commitments',
 'FY2012_13_Actual Disbursements', 'FY2012_13_Planned Commitments', 'FY2012_13_Planned disbursements', 'FY2013_14_Actual Commitments',
 'FY2013_14_Actual Disbursements', 'FY2013_14_Planned Commitments', 'FY2013_14_Planned disbursements', 'FY2014_15_Actual Commitments',
 'FY2014_15_Actual Disbursements', 'FY2014_15_Planned Commitments', 'FY2014_15_Planned disbursements', 'FY2015_16_Actual Commitments',
 'FY2015_16_Actual Disbursements', 'FY2015_16_Planned Commitments', 'FY2015_16_Planned disbursements', 'Total_Actual Commitments',
 'Total_Actual Disbursements', 'Total_Planned Commitments', 'Total_Planned disbursements']]
mgae = mgae[((mgae["Donor Group"] == "Banque mondiale") | (mgae["Donor Group"] == "Canada"))]
mgae["Project ID"] = mgae["Organizations and Project ID"]
mgae["Project ID"] = mgae["Project ID"].apply(lambda x: get_project_ID(x))

mgae["iati_identifier"] = mgae.apply(lambda x: get_iati_id(x), axis=1)
mgae = mgae.set_index("iati_identifier")

mgae_iati = mgae.join(iati, lsuffix="_aims", rsuffix="_iati", how="outer").fillna("NOT FOUND").sort_index()

In [10]:
mgae_iati.to_excel("mgae_iati_20160326.xlsx", encoding='utf-8')